# Introduction: 

## Context:

In our last lesson, we focused extensively on learning how the Naive Bayes algorithm works from a theoretical standpoint (more specifically, we learned about the multinomial Naive Bayes algorithm). 

In this guided project, we're going to study the practical side of the algorithm by building a spam filter for SMS messages.

To classify messages as spam or non-spam, we saw in the previous lesson that the computer:

1. Learns how humans classify messages.
2. Uses that human knowledge to estimate probabilities for new messages — probabilities for spam and non-spam.
3. Classifies a new message based on these probability values — if the probability for spam is greater, then it classifies the message as spam. Otherwise, it classifies it as non-spam (if the two probability values are equal, then we may need a human to classify the message).

So our first task is to "teach" the computer how to classify messages. To do that, we'll use the multinomial Naive Bayes algorithm along with a dataset of 5,572 SMS messages that are already classified by humans.

The dataset was put together by Tiago A. Almeida and José María Gómez Hidalgo, and it can be downloaded from the [The UCI Machine Learning](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection) Repository. You can also download the dataset directly from this link. The data collection process is described in more details on this page, where you can also find some of the authors' papers.

## Goal:

Our goal is to create a spam filter that classifies new messages with an accuracy greater than 80% — so we expect that more than 80% of the new messages will be classified correctly as spam or ham (non-spam).

# Part 1: Read Data

In [1]:
import pandas as pd

SMSspamdf = pd.read_csv('SMSSpamCollection',sep='\t',names=['Label','SMS'])

Find how many rows and columns data has:

In [2]:
SMSspamdf.head(10)

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...
6,ham,Even my brother is not like to speak with me. ...
7,ham,As per your request 'Melle Melle (Oru Minnamin...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...


In [3]:
SMSspamdf.shape

(5572, 2)

Find what percentage of the messages is spam and what percentage is ham ("ham" means non-spam):

In [4]:
SMSspamdf['Label'].value_counts(normalize=True)*100

ham     86.593683
spam    13.406317
Name: Label, dtype: float64

Dataset has 5500 rows, and only two columns; the label or target label we are after "Label" and the actual SMS. Looking at the relative frequencies, 4/5 of the messages are not spam, or ham, and 1/5 of the messages have been labeled as spam at this time.

# Part 2: Splitting dataset; Train and Test sets

It's very helpful to first think of a way of testing how well it works. 

When creating software (a spam filter is software), a good rule of thumb is that designing the test comes before creating the software. If we write the software first, then it's tempting to come up with a biased test just to make sure the software passes it.

Once our spam filter is done, we'll need to test how good it is with classifying new messages. To test the spam filter, we're first going to split our dataset into two categories:

- A **training set**, which we'll use to "train" the computer how to classify messages.
- A **test set**, which we'll use to test how good the spam filter is with classifying new messages.

We're going to keep 80% of our dataset for training, and 20% for testing.

- The training set will have 4,458 messages (about 80% of the dataset).
- The test set will have 1,114 messages (about 20% of the dataset).

In [5]:
# Start by randomizing the entire dataset
# frac=1 parameter to randomize the entire dataset.
# random_state=1 parameter to make sure your results are reproducible.
randomized = SMSspamdf.sample(frac=1,random_state=1)

In [6]:
# Calculate index for split
training_test_index = round(len(randomized) * 0.8)

# Training/Test split
training_set = randomized[:training_test_index].reset_index(drop=True)
test_set = randomized[training_test_index:].reset_index(drop=True)

Find the percentage of spam and ham in both the training and the test set. Are the percentages similar to what we have in the full dataset?

In [7]:
print(training_set.shape)
print(test_set.shape)

(4458, 2)
(1114, 2)


In [8]:
training_set['Label'].value_counts(normalize=True)*100

ham     86.54105
spam    13.45895
Name: Label, dtype: float64

In [9]:
test_set['Label'].value_counts(normalize=True)*100

ham     86.804309
spam    13.195691
Name: Label, dtype: float64

Percentages are similar to full dataset.

# Part 3: Math background & Data cleaning 

### Mathematical Background:

Recall from the previous lesson that when a new message comes in, our Naive Bayes algorithm will make the classification based on the results it gets to these two equations:

\begin{equation}
P(Spam | w_1,w_2, ..., w_n) \propto P(Spam) \cdot \prod_{i=1}^{n}P(w_i|Spam)
\end{equation}

\begin{equation}
P(Ham | w_1,w_2, ..., w_n) \propto P(Ham) \cdot \prod_{i=1}^{n}P(w_i|Ham)
\end{equation}

Also, to calculate P(wi|Spam) and P(wi|Ham) inside the formulas above, recall that we need to use these equations:

\begin{equation}
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}}
\end{equation}


\begin{equation}
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
\end{equation}


Lets also sumamrize what the terms in the equations above mean:

\begin{aligned}
&N_{w_i|Spam} = \text{the number of times the word } w_i \text{ occurs in spam messages} \\
&N_{w_i|Spam^C} = \text{the number of times the word } w_i \text{ occurs in non-spam messages} \\
\\
&N_{Spam} = \text{total number of words in spam messages} \\
&N_{Spam^C} = \text{total number of words in non-spam messages} \\
\\
&N_{Vocabulary} = \text{total number of words in the vocabulary} \\
&\alpha = 1 \ \ \ \ (\alpha \text{ is a smoothing parameter})
\end{aligned}


To calculate all these probabilities, we'll first need to perform a bit of data cleaning to bring the data in a format that will allow us to extract easily all the information we need.

### Data Cleaning:

##### Data currently: 

![before data](https://dq-content.s3.amazonaws.com/433/cpgp_dataset_1.png)

##### Format we want: 

![Image](https://dq-content.s3.amazonaws.com/433/cpgp_dataset_2.png)

**All words in the vocabulary are in lower case, so "SECRET" and "secret" come to be considered to be the same word.**

**Punctuation is not taken into account anymore (for instance, we can't look at the table and conclude that the first message initially had three exclamation marks).**

In [10]:
training_set.head()

,Label,SMS
0,ham,"Yep, by the pretty sculpture"
1,ham,"Yes, princess. Are you going to make me moan?"
2,ham,Welp apparently he retired
3,ham,Havent.
4,ham,I forgot 2 ask ü all smth.. There's a card on ...


In [11]:
#remove all punctuation from SMS column
training_set['SMS'] = training_set['SMS'].str.replace('\W',' ')
training_set['SMS'] = training_set['SMS'].str.lower()

In [12]:
training_set.head()

,Label,SMS
0,ham,yep by the pretty sculpture
1,ham,yes princess are you going to make me moan
2,ham,welp apparently he retired
3,ham,havent
4,ham,i forgot 2 ask ü all smth there s a card on ...


### Creation of Vocabulary Set:

In [13]:
# creation of blank list
vocabulary = []

# iterate over each row, which is a list of words, then iterate each list
# append words to vocabulary

training_set['SMS'] = training_set['SMS'].str.split()
for row in training_set['SMS']:
    for word in row:
        vocabulary.append(word)

#de-dup words
vocabulary = list(set(vocabulary))
len(vocabulary)

7783

In [14]:
word_counts_per_sms = {unique_word: [0] * len(training_set['SMS']) for unique_word in vocabulary}

for index, sms in enumerate(training_set['SMS']):
    for word in sms:
        word_counts_per_sms[word][index] += 1

word_counts_df = pd.DataFrame(word_counts_per_sms)

In [15]:
#concat dataframe with training

training_word_count_df = pd.concat([training_set,word_counts_df],axis=1)

In [16]:
training_word_count_df.head()

,Label,SMS,0,00,000,000pes,008704050406,0089,01223585334,02,...,zindgi,zoe,zogtorius,zouk,zyada,é,ú1,ü,〨ud,鈥
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0


Done cleaning data.

# Part 4: Creation of Spam Filter

Recall and refer to part #3 in the notebook for the mathematical background, which describes the algorithm we want to create here. Lets calculate the naive bayes algorithm down into parts and bring it altogeher;

 - P(Spam) & P(Ham)
 - N_spam, N_ham, & N_vocabulary 
 
where:

- N_spam is equal to the number of words in all the spam messages — it's not equal to the number of spam messages, and it's not equal to the total number of unique words in spam messages.

- N_ham is equal to the number of words in all the non-spam messages — it's not equal to the number of non-spam messages, and it's not equal to the total number of unique words in non-spam messages.

In [17]:
# probability of P(Spam) and P(Ham)

# P(Spam)
p_ham = training_word_count_df['Label'].value_counts(normalize=True)['ham']

# P(Ham)
p_spam = training_word_count_df['Label'].value_counts(normalize=True)['spam']

In [18]:
#n_ham
n_ham = 0
for row in training_word_count_df.loc[training_word_count_df['Label']=='ham','SMS']:
    n_ham += len(row)

#n_spam
n_spam = 0
for row in training_word_count_df.loc[training_word_count_df['Label']=='spam','SMS']:
    n_spam += len(row)

#n_vocab
n_vocabulary = len(vocabulary)

#laplace smoothing
alpha = 1 
print('N_ham: {:,d}'.format(n_ham))
print('N_spam: {:,d}'.format(n_spam))
print('N_vocab: {:,d}'.format(n_vocabulary))
print('Alpha: {}'.format(alpha))

N_ham: 57,237
N_spam: 15,190
N_vocab: 7,783
Alpha: 1


In [19]:
#intialize parameters
wi_spam = {key: 0 for key in vocabulary}
wi_ham  = {key: 0 for key in vocabulary}

#isolating ham and spam dataframes
ham_df = training_word_count_df.loc[training_word_count_df['Label']=='ham',:]
spam_df = training_word_count_df.loc[training_word_count_df['Label']=='spam',:]

In [22]:
# iterate over the vocabulary, and, for each word, 
# calculate P(wi|Spam) and P(wi|Ham) using the formulas we mentioned above.

# P(wi|Spam)
# P(wi|Ham)
for v in vocabulary:
    n_wi_spam = spam_df[v].sum()
    n_wi_ham = ham_df[v].sum()
    
    p_wi_spam = (n_wi_spam+alpha)/(n_wi_spam+alpha*n_vocabulary)
    p_wi_ham = (n_wi_ham+alpha)/(n_wi_ham+alpha*n_vocabulary)
    
    wi_spam[v] = p_wi_spam
    wi_ham[v] = n_wi_ham

In [25]:
# creation of classification algorithm
import re

def classify(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    '''    
    This is where we calculate:

    p_spam_given_message = ?
    p_ham_given_message = ?
    '''    
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    
    for word in message:
        if word in wi_spam:
            p_spam_given_message *= wi_spam[word]
        if word in wi_ham:
            p_ham_given_message *= wi_ham[word]
            
    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)

    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal proabilities, have a human classify this!')

In [27]:
classify('WINNER!! This is the secret code to unlock the money: C3421.')

P(Spam|message): 1.9931640894966366e-21
P(Ham|message): 0.0
Label: Spam


In [28]:
classify("Sounds good, Tom, then see u there")

P(Spam|message): 4.651925051800531e-22
P(Ham|message): 10783731720095.871
Label: Ham


Pretty awesome to see how Naive Bayes algorithm is working on these two test cases. Lets now test on our larger training set.

# Part 5: Training our Algorithm

First off, we'll change the classify() function that we wrote previously to return the labels instead of printing them.

In [31]:
def classify_test_set(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    p_spam_given_message = p_spam
    p_ham_given_message = p_ham

    for word in message:
        if word in wi_spam:
            p_spam_given_message *= wi_spam[word]

        if word in wi_ham:
            p_ham_given_message *= wi_ham[word]

    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_spam_given_message > p_ham_given_message:
        return 'spam'
    else:
        return 'needs human classification'

In [49]:
test_set['predicted'] = test_set['SMS'].apply(classify_test_set)
test_set.head()

,Label,SMS,predicted
0,ham,Later i guess. I needa do mcat study too.,ham
1,ham,But i haf enuff space got like 4 mb...,ham
2,spam,Had your mobile 10 mths? Update to latest Oran...,spam
3,ham,All sounds good. Fingers . Makes it difficult ...,ham
4,ham,"All done, all handed in. Don't know if mega sh...",spam


# Part 6: Testing Accuracy

We will measure accuracy of our algorithm using the formula:

\begin{equation}
\text{Accuracy} = \frac{\text{number of correctly classified messages}}{\text{total number of classified messages}}
\end{equation}


In [69]:
#initialize correct
correct = (test_set['Label']==test_set['predicted']).sum()

#initialize total
total = test_set['SMS'].count()

accuracy = correct/total
print(accuracy*100)

95.60143626570917


Accuracy is good, but i expected better. Somewhere in my code i probably messed up but for raw coding a Naive Baiyes algorithm 95% is pretty good.